In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import pandas as pd
import numpy as np
import larch.numba as lx
from larch import P, X

from numpy.random import default_rng

/mnt/ssd_data/ed_data/dev/larch/larch/numba/model.py:23: UserWarning: 

### larch.numba is experimental, and not feature-complete ###
 the first time you import on a new system, this package will
 compile optimized binaries for your machine, which may take 
 a little while, please be patient 

  warnings.warn( ### EXPERIMENTAL ### )


In [3]:
hh, pp, tour, skims, emp = lx.example(200, ['hh', 'pp', 'tour', 'skims', 'emp'])

In [4]:
#logsums_file = lx.example(202, output_file='logsums.zarr.zip')
#logsums = lx.DataArray.from_zarr('logsums.zarr.zip')

In [5]:
# ca = lx.Dataset.construct(
# #     {'logsum': logsums},
# #     caseid='TOURID',
# #     alts=skims.TAZ_ID,
# # )

# # tree = lx.DataTree(
# #     base=ca,
# #     tour=tour.rename_axis(index='TOUR_ID'),
# #     hh=hh.set_index("HHID"),
# #     #person=pp.set_index('PERSONID'),
# #     emp=emp,
# #     skims=lx.Dataset.construct.from_omx(skims),
# #     relationships=(
# #         "base.TAZ_ID @ emp.TAZ",
# #         "base.TOURID -> tour.TOUR_ID",
# #         "tour.HHID @ hh.HHID",
# #         #"tour.PERSONID @ person.PERSONID",
# #         "hh.HOMETAZ @ skims.otaz",
# #         "base.TAZ_ID @ skims.dtaz"
# #     ),
# # )
# m = lx.Model(datatree=tree)
# m.title = "blah"
# m.quantity_ca = (
#         #+ P.EmpRetail_HighInc * X('RETAIL_EMP * (INCOME>50000)')
#         #+ P.EmpNonRetail_HighInc * X('NONRETAIL_EMP') * X("INCOME>50000")
#         #+ P.EmpRetail_LowInc * X('RETAIL_EMP') * X("INCOME<=50000")
#         #+ P.EmpNonRetail_LowInc * X('NONRETAIL_EMP') * X("INCOME<=50000")
# )

# m.quantity_scale = P.Theta
# m.utility_ca = (
#     + P.logsum * X.logsum
#     + P.distance * X.AUTO_DIST
# )

# m.choice_co_code = "tour.DTAZ"
# m.lock_values(
#     EmpRetail_HighInc=0,
#     EmpRetail_LowInc=0,
# )
# #print(f"init ll = {m.loglike()}")
# #m.maximize_loglike()

In [6]:
tour['test_code'] = tour.DTAZ.astype(str) + "_" + tour.TOURPURP.astype(str)
all_alternatives = [f"{dest}_{purp}" for dest in range(tour.DTAZ.min(), tour.DTAZ.max()+1) for purp in range(tour.TOURPURP.min(), tour.TOURPURP.max()+1)]

# looks like strings are not allowed, also larch needs 1-based indexes because nest root is 0 I think
alt_map = {k: v+1 for v, k in enumerate(all_alternatives)}
all_alts_int = list(map(lambda x: alt_map[x], all_alternatives))

#observations = lx.Dataset.construct(tour.set_index("TOURID"), caseid="TOURID", alts=all_alternatives)
tour['test_code'] = tour['test_code'].map(alt_map)

#### might need to make sure that the chosen alternative has larger attractions for equal TAZ

group by taz, get ratio of 1 to 2, then multiply total attractions of TAZs with that ratio. doesn't work. Let's just simulate choice with a pre-determined utility fct

In [7]:
# ## attractions. Let's say purp 1 is NONRETAIL_EMP and purp 2 is RETAIL_EMP
# purp_1_attr = emp.reset_index()[["TAZ", "NONRETAIL_EMP"]].rename(columns={"NONRETAIL_EMP": "attractions"}).copy()
# purp_1_attr["purpose"] = 1
# purp_1_attr["altid"] = purp_1_attr.TAZ.astype(str) + "_" + purp_1_attr.purpose.astype(str)

# purp_2_attr = emp.reset_index()[["TAZ", "RETAIL_EMP"]].rename(columns={"RETAIL_EMP": "attractions"}).copy()
# purp_2_attr["purpose"] = 2
# purp_2_attr["altid"] = purp_2_attr.TAZ.astype(str) + "_" + purp_2_attr.purpose.astype(str)

# attrs = pd.concat([purp_1_attr, purp_2_attr], ignore_index=True)
# attrs["altid"] = attrs["altid"].map(alt_map)

In [8]:
num_purp_obs_per_zone = tour.groupby(["DTAZ"]).TOURPURP.value_counts().to_frame("num_obs").reset_index()
# if no missing could sort by DTAZ and TOURPURP and then do a window divide
attr_ratio = num_purp_obs_per_zone.groupby(["DTAZ"]).apply(lambda x: x.loc[x.TOURPURP == 1].num_obs.sum() / x.loc[x.TOURPURP == 2].num_obs.sum()).to_frame("ratio_1_over_2")

attrs = emp[["TOTAL_EMP"]].join(attr_ratio).reset_index()
attrs["attractions_2"] = attrs["TOTAL_EMP"] * attrs["ratio_1_over_2"]

purp_1_attr = attrs[["TAZ", "TOTAL_EMP"]].rename(columns={"TOTAL_EMP": "attractions"}).copy()
purp_1_attr["purpose"] = 1
purp_1_attr["altid"] = purp_1_attr.TAZ.astype(str) + "_" + purp_1_attr.purpose.astype(str)
purp_2_attr = attrs[["TAZ", "attractions_2"]].rename(columns={"attractions_2": "attractions"}).copy()
purp_2_attr["purpose"] = 2
purp_2_attr["altid"] = purp_2_attr.TAZ.astype(str) + "_" + purp_2_attr.purpose.astype(str)

attrs = pd.concat([purp_1_attr, purp_2_attr], ignore_index=True)
attrs["altid"] = attrs["altid"].map(alt_map)

assert not attrs.loc[attrs.attractions == 0].shape[0]

In [9]:
tour = tour.merge(hh[["HHID", "HOMETAZ"]], on="HHID")

In [10]:
auto_dist = np.array(skims["AUTO_DIST"])
dist_obs_dest = auto_dist[tour.HOMETAZ.values - 1,:]
display(dist_obs_dest.shape)
dist_obs_dest_purp = np.tile(dist_obs_dest, 2)
display(dist_obs_dest_purp.shape)

attr_dest_purp = np.log(attrs.sort_values(by=["TAZ", "purpose"]).attractions.values)
display(attr_dest_purp.shape)

(20739, 40)

(20739, 80)

(80,)

In [11]:
rng = default_rng(seed=234310)
utils = -0.01 * dist_obs_dest_purp + attr_dest_purp - np.log(-np.log(rng.uniform(size=dist_obs_dest_purp.shape)))
chosen_vals = utils.argmax(axis=1)

# these -1 are by definition equal to tour alts above because we ordered by taz,purp
tour["chosen"] = chosen_vals + 1

In [12]:
tour.head()

,TOURID,HHID,PERSONID,DTAZ,TOURMODE,TOURPURP,N_STOPS,N_TRIPS,N_TRIPS_HBW,N_TRIPS_HBO,N_TRIPS_NHB,test_code,HOMETAZ,chosen
0,0,50000,60000,22,1,1,0,2,2,0,0,43,22,80
1,1,50000,60001,4,1,1,0,2,2,0,0,7,22,78
2,2,50000,60001,10,2,2,0,2,0,2,0,20,22,49
3,3,50000,60002,20,1,1,0,2,2,0,0,39,22,39
4,4,50000,60002,20,1,2,1,3,0,2,1,40,22,2


In [13]:
tour.head()

,TOURID,HHID,PERSONID,DTAZ,TOURMODE,TOURPURP,N_STOPS,N_TRIPS,N_TRIPS_HBW,N_TRIPS_HBO,N_TRIPS_NHB,test_code,HOMETAZ,chosen
0,0,50000,60000,22,1,1,0,2,2,0,0,43,22,80
1,1,50000,60001,4,1,1,0,2,2,0,0,7,22,78
2,2,50000,60001,10,2,2,0,2,0,2,0,20,22,49
3,3,50000,60002,20,1,1,0,2,2,0,0,39,22,39
4,4,50000,60002,20,1,2,1,3,0,2,1,40,22,2


In [14]:
attrs[["attractions", "altid", "TAZ", "purpose"]].set_index('altid').head()

,attractions,TAZ,purpose
altid,,,
1,458.0,1,1
3,91.0,2,1
5,199.0,3,1
7,285.0,4,1
9,57.0,5,1


In [15]:
np.array(skims.AUTO_DIST).shape

(40, 40)

In [18]:
skims

<larch.OMX> ⋯/exampville_skims.omx
 |  shape:(40, 40)
 |  data:
 |    AUTO_COST    (float64)
 |    AUTO_DIST    (float64)
 |    AUTO_TIME    (float64)
 |    BIKE_TIME    (float64)
 |    TRANSIT_FARE (float64)
 |    TRANSIT_IVTT (float64)
 |    TRANSIT_OVTT (float64)
 |    WALK_DIST    (float64)
 |    WALK_TIME    (float64)
 |  lookup:
 |    TAZ_AREA_TYPE (40 |S3)
 |    TAZ_ID        (40 int64)

In [17]:
attrs

,TAZ,attractions,purpose,altid
0,1,458.000000,1,1
1,2,91.000000,1,3
2,3,199.000000,1,5
3,4,285.000000,1,7
4,5,57.000000,1,9
...,...,...,...,...
75,36,189.490842,2,72
76,37,13.716216,2,74
77,38,70.447020,2,76
78,39,158.378378,2,78


In [21]:
lx.Dataset.construct(tour.set_index("TOURID"), caseid="TOURID", alts=all_alts_int)

<xarray.Dataset>
Dimensions:      (TOURID: 20739, _altid_: 80)
Coordinates:
  * TOURID       (TOURID) int64 0 1 2 3 4 5 ... 20734 20735 20736 20737 20738
  * _altid_      (_altid_) int64 1 2 3 4 5 6 7 8 9 ... 73 74 75 76 77 78 79 80
Data variables: (12/13)
    HHID         (TOURID) int64 50000 50000 50000 50000 ... 54999 54999 54999
    PERSONID     (TOURID) int64 60000 60001 60001 60002 ... 72345 72346 72347
    DTAZ         (TOURID) int64 22 4 10 20 20 25 35 25 ... 1 16 4 40 36 5 28 11
    TOURMODE     (TOURID) int64 1 1 2 1 1 1 1 1 1 1 1 ... 1 1 5 1 1 1 1 5 1 1 2
    TOURPURP     (TOURID) int64 1 1 2 1 2 2 2 1 2 2 1 ... 1 1 2 1 2 2 1 2 1 2 2
    N_STOPS      (TOURID) int64 0 0 0 0 1 0 0 2 2 0 1 ... 2 0 1 0 4 3 0 1 3 4 1
    ...           ...
    N_TRIPS_HBW  (TOURID) int64 2 2 0 2 0 0 0 1 0 0 1 ... 1 2 0 2 0 0 2 0 1 0 0
    N_TRIPS_HBO  (TOURID) int64 0 0 2 0 2 2 2 1 2 2 1 ... 1 0 2 0 2 2 0 2 1 2 2
    N_TRIPS_NHB  (TOURID) int64 0 0 0 0 1 0 0 2 2 0 1 ... 2 0 1 0 4 3 0 1 3 4 1
    test_code    (TOURID) int64 43 7 20 39 40 50 70 49 ... 1 32 8 79 72 9 56 22
    HOMETAZ      (TOURID) int64 22 22 22 22 22 22 22 22 ... 14 14 14 14 14 14 14
    chosen       (TOURID) int64 80 78 49 39 2 79 7 80 ... 36 80 23 43 21 77 76
Attributes:
    _caseid_:  TOURID
    _altid_:   _altid_

In [19]:
tree = lx.DataTree(
    obs=lx.Dataset.construct(tour.set_index("TOURID"), caseid="TOURID", alts=all_alts_int),
    attr=attrs[["attractions", "altid", "TAZ", "purpose"]].set_index('altid'),
    skims=lx.Dataset.construct.from_omx(skims),
    relationships=(
        "obs._altid_ @ attr.altid",
        "obs.HOMETAZ @ skims.otaz",
        ## link the alternative set to the dtaz, not the chosen alternative, so don't do "obs.DTAZ @ skims.dtaz", but:
        "attr.TAZ @ skims.dtaz",
    ),
)

# tree = lx.DataTree(
#     obs=lx.Dataset.construct(tour.set_index("TOURID"), caseid="TOURID", alts=np.arange(tour.DTAZ.min(), tour.DTAZ.max()+1)),
#     attr=emp,
#     hh=hh.set_index("HHID"),
#     skims=lx.Dataset.construct.from_omx(skims),
#     relationships=(
#         "obs._altid_ @ attr.TAZ",
#         "tour.HHID @ hh.HHID",
#         "hh.HOMETAZ @ skims.otaz",
#         "obs.DTAZ @ skims.dtaz",
#     ),
# )

In [ ]:
#Theta * log( exp(param_zero) * attractions)

In [20]:
m = lx.Model(datatree=tree)
m.title = "blah"

m.quantity_ca = P("zero") * X("attractions")
m.quantity_scale = P.Theta

m.utility_ca = P.distance * X.AUTO_DIST + P.log_distance * X('np.log(AUTO_DIST)') #+ P.purpose_is_one * X("purpose == 1")

m.choice_co_code = "obs.chosen"

m.availability_var = "attr.attractions > 0"

m.lock_values(
    zero=0,
    Theta=1.,
    #distance=-10.0
)
m.set_cap(10)

#m.pf.loc[m.pf.index == "distance", "initvalue"] = -0.5
#m.pf.loc[m.pf.index == "distance", "value"] = -0.5

#init ll = -94714.36502628266
# -84048.12422143703
print(f"init ll = {m.loglike()}")
m.maximize_loglike(maxiter=1000, tol=1e-12)

init ll = -83824.5336170936


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
Theta,1.000000,1.0,1.0,1.0,1.0,1,,1.000000
distance,-0.006289,0.0,0.0,-10.0,10.0,0,,-0.006289
log_distance,0.025538,0.0,0.0,-10.0,10.0,0,,0.025538
zero,0.000000,0.0,0.0,0.0,0.0,1,,0.000000


┣                x: Theta           1.000000
┃                   distance       -0.006289
┃                   log_distance    0.025538
┃                   zero            0.000000
┃                   dtype: float64
┣          loglike: -83824.2976330108
┣        d_loglike: Theta                NaN
┃                   distance       -0.001953
┃                   log_distance   -0.000000
┃                   zero                 NaN
┃                   dtype: float64
┣              nit: 5
┣             nfev: 33
┣             njev: 5
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=2, microseconds=354897)
┣           method: 'slsqp'
┣          n_cases: 20739
┣ iteration_number: 5
┣          logloss: 4.041867864073041

In [ ]:
#distance does not change things?

In [ ]:
m.probability()

In [ ]:
m.dataflows